# Comment
* p = pinyin
* c = character
* o = order

# Todo
1. [OK] Rework priority 
2. [OK] fill in missing solo
3. [OK] split concerto
4. [OK] ui, ve problem
5. [OK] dict order
6. [OK] zhuyin alignment for 1 sound


* dict included: 18.8M
* ui, ve       : 18.9M
* add zi2pin   : 20.4M

In [1]:
import sqlite3
import time
import numpy as np
from colorama import Fore

In [2]:
conn = sqlite3.connect('magician20130911.db3')
c = conn.cursor()
(list(c.execute('PRAGMA synchronous=OFF')),
list(c.execute('PRAGMA locking_mode=EXCLUSIVE')),
list(c.execute('PRAGMA temp_store=MEMORY')),
list(c.execute('PRAGMA mmap_size=100000000')),
list(c.execute('PRAGMA journal_mode=OFF')))

([], [('exclusive',)], [], [(100000000,)], [('off',)])

In [3]:
# newDB = sqlite3.connect(':memory:')
newDB = sqlite3.connect('v5.db')
n = newDB.cursor()

In [4]:
list(n.execute('PRAGMA synchronous')), list(n.execute('PRAGMA locking_mode'))

([(2,)], [('normal',)])

In [5]:
list(n.execute('PRAGMA synchronous=OFF')), list(n.execute('PRAGMA locking_mode=EXCLUSIVE'))
list(n.execute('PRAGMA synchronous')), list(n.execute('PRAGMA locking_mode'))

([(0,)], [('exclusive',)])

In [6]:
list(n.execute('PRAGMA temp_store')), list(n.execute('PRAGMA mmap_size'))

([(0,)], [(0,)])

In [7]:
list(n.execute('PRAGMA temp_store=MEMORY')), list(n.execute('PRAGMA mmap_size=100000000'))
list(n.execute('PRAGMA temp_store')), list(n.execute('PRAGMA mmap_size'))

([(2,)], [(100000000,)])

In [8]:
list(n.execute('PRAGMA journal_mode')), list(n.execute('PRAGMA journal_mode=OFF')), list(n.execute('PRAGMA journal_mode'))

([('delete',)], [('off',)], [('off',)])

# Create Table

In [9]:
# n.execute("drop table meta")
n.execute("create table 'meta'(k text collate binary, v text collate binary, primary key (k collate binary) " + \
          "on conflict abort) without rowid")
n.execute("insert into meta values(?, ?)", ('version', 5))
n.execute("insert into meta values(?, ?)", ('time', time.strftime('%d %b %Y %H:%M:%S')))

In [10]:
for i in range(1, 10):
    sql = 'create table "s' + str(i) + '"(p text collate binary, c text collate binary, o integer not null, ' + \
        'primary key (p collate binary, c collate binary, o) on conflict ignore) without rowid'
    n.execute(sql)
newDB.commit()

In [11]:
# for i in n.execute('SELECT * FROM sqlite_master;'):
#     print(i)

# Rework priority

In [12]:
m = [('solo', 's1', '16'), ('duet', 's2', '16'), ('trio', 's3', '8'), ('quartet', 's4', '6'), ]

In [13]:
for oldTableName, newTableName, limit in m:
    maxCount = 0
    pinyinList = list(c.execute('select distinct pinyin from ' + oldTableName))
    print(len(pinyinList), end='\t')
    for pinyin in pinyinList:
        x = list(c.execute('select * from ' + oldTableName + ' where pinyin=? order by priority desc limit ' + limit, pinyin))
        maxCount = max(len(x), maxCount)
        n.executemany('INSERT INTO ' + newTableName + ' VALUES (?,?,?)', [(j[0], j[1], 500+i*2) for i, j in enumerate(x)])
    print(maxCount)
newDB.commit()

369	16
50465	16
183693	8
169822	6


In [14]:
for i in range(10, 20, 2):
    pinyin = list(c.execute('select distinct pinyin from concerto where length(pinyin)='+str(i)))
    length = i // 2
    print(length, len(pinyin))
    for p in pinyin:
        x = list(c.execute('select * from concerto where pinyin=? order by priority desc limit 4', p))
        n.executemany('INSERT INTO s' + str(length) + ' VALUES (?,?,?)', [(j[0], j[1], 500+i*2) for i, j in enumerate(x)])

5 96750
6 45900
7 24991
8 15325
9 9621


# Fix ie/ue problem

In [15]:
def isE(p, z):
    try:
        cursor = c.execute('select * from dict where pinyin glob ? and zi glob ?', (p, '*' + z + '*'))
        dic = next(cursor)
        zhuyin = dic[1]
        return 'ㄝ' in zhuyin
    except: 
        print('warn', p, z)
        return False
isE('Nr', '缺'), isE('Lr', '為')

(True, False)

In [16]:
def fix(table, iZiTarget):
    globPinyin = ['?'] * (2 * table)
    globPinyin[2 * iZiTarget + 1] = 'r'
    globPinyin = ''.join(globPinyin)
    for i in list(n.execute(
            'select * from s' + str(table) + ' where p glob ? order by o desc', (globPinyin,))):
        pinyin, zi = i[:2]
        pinyinI, ziI = pinyin[iZiTarget*2:iZiTarget*2+2], zi[iZiTarget]
        
        if isE(pinyinI, ziI):
            newPinyin = pinyin[:2 * iZiTarget + 1] + 'd' + pinyin[2 * iZiTarget + 2:]
            sql = 'update s' + str(table) + ' set p = ? where c = ? and p = ?'
            n.execute(sql, (newPinyin, zi, pinyin))
# fix(3, 2)

In [17]:
for i in range(1, 10):
    for j in range(i):
        print(i, j)
        fix(i, j)

1 0
warn Hr 摾
2 0
2 1
3 0
3 1
warn Ir 繪
3 2
warn Ir 繪
4 0
4 1
warn Ir 繪
4 2
warn Ir 繪
warn Ir 繪
4 3
warn Ir 繪
5 0
5 1
5 2
warn Ir 繪
5 3
warn Ir 繪
5 4
warn Ir 繪
6 0
6 1
6 2
6 3
6 4
6 5
7 0
7 1
7 2
7 3
7 4
7 5
7 6
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8


# Dict

In [18]:
sql = 'create table "d"(p text collate binary, o integer not null, c text collate binary, z text collate binary, ' + \
        'primary key (p collate binary, o) on conflict abort) without rowid'
n.execute(sql);

In [19]:
pinyin = list(c.execute('select distinct pinyin from dict'))

In [20]:
trans = {ord(i): j for i, j in zip('ˊˇˋ˙', '2345')}
def key(s):
    return s.translate(trans)
key('ㄅㄚˇ')

'ㄅㄚ3'

In [21]:
for p in pinyin:
    select = list(c.execute('select pinyin, priority, zi, orig from dict where pinyin=? and zi<>orig and zi<>"ㄉˊ"', p))
    select.sort(key=lambda x: key(x[3]))
    select = [list(i) for i in select]
#     print(select)
    for i, j in enumerate(select):
        j[1] = i
        if not any(k in j[3] for k in 'ˊˇˋ˙'):
            j[3] = j[3] + '　'
#     print(select)
    n.executemany('insert into d values(?,?,?,?)', select)

# Fix ie/ue problem

In [22]:
for i in list(n.execute("select * from d where p glob '?r' and z glob '*ㄝ*'")):
    newPinyin = i[0].replace('r', 'd')
    offset = next(n.execute("select o from d where p = ? order by o desc", (i[0].replace('r', 'd'),)))[0]
    offset += 1
    sql = "update d set p = ?, o = ? where p = ? and z = ?"
    args = (newPinyin, offset, i[0], i[3])
    n.execute(sql, args)

# Add solo less than 16

In [23]:
from itertools import zip_longest
def offset(p):
    return next(n.execute("select o from s1 where p = ? order by o desc", (p,)))[0] + 1

for i in list(n.execute("select distinct p from s1")):
    p = i[0]
    j = list(n.execute("select * from s1 where p=? order by o", (p,)))
    if len(j) < 16:
#         print((j))
        print('\n', p, end='')
        zSet = {i[1] for i in j}
#         print(zSet)
        d = list(n.execute("select c from d where p=?", (p,)))
#         print(d)
        dList = [i[0] for i in d]
#         print(dList)
        dList2 = []
        for l in zip_longest(*dList):
            dList2.extend(l)
        dList2 = [i for i in dList2 if i is not None]
#         print(dList2)
#         print(offset(p))
        while offset(p) < 16:
            sql = "insert into s1 values(?,?,?)"
            c = None
            for k in dList2:
                if k not in zSet:
                    c = k
                    zSet.add(k)
                    break
            if c is None: break
            arg = (p, c, offset(p))
            n.execute(sql, arg)
            print(c, end='')
#         break


 Ad
 Ae
 Aj
 Al
 An
 Ap
 As
 Av
 Ba
 Bd
 Be
 Bh
 Bj
 Bk
 Bl
 Bm
 Bn
 Bp
 Bs
 Bu
 Bv
 Cc
 Cd
 Ce
 Ch
 Ci
 Cj
 Ck
 Cm
 Cp
 Ct
 Da
 Db
 Dh
 Dj
 Dk
 Dl
 Dp
 Ds
 Ea
 Ec
 Ef
 Ei
 Ek
 El
 En
 Er
 Et
 Fb
 Fc
 Fd
 Fg
 Fh
 Fl
 Fm
 Fn
 Fr
 Fs
 Fv
 Gb
 Gc
 Gd
 Ge
 Gf
 Gg
 Gh
 Gj
 Gk
 Gl
 Gm
 Gn
 Go
 Gp
 Gs
 Gt
 Gv
 Gw
 Ha
 Hb
 Hc
 He
 Hg
 Hi
 Hk
 Hl
 Hm
 Ho
 Hr
 Ia
 Ie
 If
 Ig
 Ii
 Ij
 Ik
 Il
 Im
 In
 Io
 Iq
 Ir
 Ja
 Jb
 Je
 Jg
 Jh
 Ji
 Jj
 Jk
 Jl
 Jm
 Jn
 Jo
 Jp
 Jq
 Jv
 Ka
 Kc
 Ke
 Kf
 Kh
 Kj
 Kk
 Kl
 Km
 Kn
 Kq
 Le
 Lf
 Lh
 Lj
 Lk
 Ll
 Lo
 Lq
 Lr
 Mt
 Na
 Nn
 Np
 Ns
 Nt
 Nw
 Oa
 Qa
 Qb
 Qc
 Qe
 Qf
 Qg
 Qk
 Ql
 Qm
 Qo
 Qp
 Qq
 Qr
 Rb
 Rc
 Re
 Rg
 Rh
 Ri
 Rm
 Rp
 Rq
 Rr
 Ru
 Rv
 Sb
 Sc
 Se
 Sf
 Sh
 Sk
 Sl
 Sm
 Sn
 So
 Sq
 Sr
 Tb
 Tc
 Tg
 Th
 Tj
 Tk
 Tl
 Tn
 Tr
 Tu
 Tv
 Ua
 Ub
 Uc
 Ue
 Uf
 Uh
 Ui
 Uj
 Uk
 Ul
 Um
 Up
 Ur
 Va
 Vg
 Vh
 Vj
 Vk
 Vl
 Vn
 Vp
 Vr
 Wa
 Wc
 We
 Wg
 Wh
 Wi
 Wj
 Wk
 Wl
 Wm
 Wn
 Wp
 Wr

# Zhuyin

In [24]:
n.execute('insert into s1 values("Jc", "ㄎ", 996)')
n.execute('insert into s1 values("Kc", "ㄏ", 996)')
n.execute('insert into s1 values("Su", "式", 996)')
n.execute('insert into s1 values("Qu", "隻", 996)')

# Zi2Pin

In [25]:
connZi2Pin = sqlite3.connect('zi2pin.db3')
z = connZi2Pin.cursor()

In [26]:
x = z.execute('select * from z where length(zi)=1')
x = list(x)

zi = [i[0] for i in x]

pinyin = [[i for i in xi[2:6] if i is not None] for xi in x]
pinyin = [','.join(i) for i in pinyin]

len(x), zi[:5], pinyin[:5]

(20790, ['七', '乃', '九', '二', '人'], ['Nu', 'Ge', 'Mt', 'Ll', 'Tj'])

In [27]:
n.execute("create table 'c2p'(c text collate binary, p text collate binary, primary key (c collate binary) " + \
          "on conflict abort) without rowid")

In [28]:
for i in zip(zi, pinyin):
    n.execute('insert into c2p values (?,?)', i)
newDB.commit()

In [29]:
count = 0
original = list(n.execute('select * from c2p where p glob "?r"'))
for i in original:
    if isE(i[1], i[0]):
        print(i)
        count += 1
        newPinyin = i[0][0] + 'd'
        sql = 'update c2p set p = ? where p = ? and c = ?'
        n.execute(sql, (newPinyin, i[1], i[0]))
#         break
count

warn Wr ⼢
('乴', 'Or')
('亅', 'Mr')
('倔', 'Mr')
('傕', 'Mr')
('僪', 'Mr')
('决', 'Mr')
('刔', 'Mr')
('刖', 'Lr')
('剠', 'Hr')
('劂', 'Mr')
('却', 'Nr')
('卻', 'Nr')
('厥', 'Mr')
('吙', 'Or')
('噘', 'Mr')
('噦', 'Lr')
('噱', 'Or')
('嚯', 'Gr')
('坹', 'Or')
('埆', 'Nr')
('堀', 'Mr')
('壆', 'Or')
('妜', 'Mr')
('嬳', 'Lr')
('孓', 'Mr')
('學', 'Or')
('屩', 'Mr')
('屫', 'Mr')
('岄', 'Lr')
('岤', 'Or')
('岳', 'Lr')
('峃', 'Or')
('崅', 'Nr')
('崛', 'Mr')
('嶨', 'Or')
('嶽', 'Lr')
('弡', 'Mr')
('彏', 'Mr')
('彟', 'Lr')
('怯', 'Nr')
('恱', 'Lr')
('悅', 'Lr')
('悦', 'Lr')
('悫', 'Nr')
('愨', 'Nr')
('慤', 'Nr')
('憠', 'Mr')
('憰', 'Mr')
('戄', 'Mr')
('戉', 'Lr')
('抈', 'Lr')
('抉', 'Mr')
('挗', 'Mr')
('捔', 'Mr')
('捳', 'Lr')
('掘', 'Mr')
('掠', 'Hr')
('搉', 'Nr')
('撅', 'Mr')
('擽', 'Hr')
('攫', 'Mr')
('斈', 'Or')
('曰', 'Lr')
('月', 'Lr')
('柽', 'Mr')
('桖', 'Or')
('桷', 'Mr')
('榷', 'Nr')
('樰', 'Or')
('樾', 'Lr')
('橛', 'Mr')
('欔', 'Mr')
('欮', 'Mr')
('毃', 'Nr')
('氒', 'Mr')
('決', 'Mr')
('泧', 'Lr')
('泬', 'Or')
('泶', 'Or')
('潏', 'Mr')
('澚', 'Lr')
('澩', 'Or')
('瀹', 

216

# Finalize

In [30]:
newDB.commit()
n.execute('vacuum')
newDB.close()